<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://raw.githubusercontent.com/logpai/logparser/master/logs/HDFS/HDFS_2k.log'


--2023-08-31 13:11:04--  https://raw.githubusercontent.com/logpai/logparser/master/logs/HDFS/HDFS_2k.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287848 (281K) [text/plain]
Saving to: ‘HDFS_2k.log’

HDFS_2k.log         100%[===================>] 281.10K  --.-KB/s    in 0.04s   

2023-08-31 13:11:04 (7.34 MB/s) - ‘HDFS_2k.log’ saved [287848/287848]



In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2ec907ae9d059624b9e8cbd0d50a9e0fea23b6f7f2d7b754098e0561704b68d1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import re
import string
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA

In [31]:
def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message

    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    # s = re.sub(r'(\d+\.){3}\d+(:\d+)?', " ", s)
    # s = re.sub(r'(\/.*?\.[\S:]+)', ' ', s)
    s = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in content.strip().split()])
    return s

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
text = "081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating"

pattern = r"blk_\d+"  # Regular expression pattern to match "blk_" followed by digits
match = re.search(pattern, text)
print(match)
print(match.group())

<re.Match object; span=(81, 102), match='blk_38865049064139660'>
blk_38865049064139660


In [50]:
def find_blockid(text):
  match = re.search(r"blk_[-\w]+", text)
  return match.group()

In [17]:
find_blockid(text)

'blk_38865049064139660'

In [52]:
with open('/content/HDFS_2k.log', "r") as file:

    log2index = {}    # give for each unique cleaned log an index
    index2embed = {}  # give for each index(belongs to a unique cleaned log) sentence embedding of that unique cleaned log
    blockId_logs = {} # gather logs(index) which are blong to a specific block id
    index = 0

    for line in file:

      blockId = find_blockid(line)
      if not blockId in blockId_logs:
            blockId_logs[blockId] = []

      cleaned_line = clean(line)

      if log2index.get(cleaned_line) is None:
          log2index[cleaned_line] = index
          index2embed[index] = model.encode(cleaned_line)
          blockId_logs[blockId].append(index)
          index += 1
      else :
          blockId_logs[blockId].append(index)


In [53]:
len(blockId_logs)

1994

In [54]:
blockId_logs

{'blk_38865049064139660': [0],
 'blk_-6952295868487656571': [1],
 'blk_7128370237687728475': [1],
 'blk_8229193803249955061': [2],
 'blk_-6670958622368987959': [2],
 'blk_3050920587428079149': [2],
 'blk_7888946331804732825': [2],
 'blk_2377150260128098806': [2],
 'blk_572492839287299681': [2],
 'blk_3587508140051953248': [2],
 'blk_5402003568334525940': [3],
 'blk_5792489080791696128': [3],
 'blk_1724757848743533110': [4],
 'blk_8015913224713045110': [4],
 'blk_-5623176793330377570': [4],
 'blk_-1727475099218615100': [4],
 'blk_5017373558217225674': [5],
 'blk_9212264480425680329': [5],
 'blk_-7878121102358435702': [5],
 'blk_4568434182693165548': [5],
 'blk_-5704899712662113150': [5],
 'blk_-4794867979917102672': [5],
 'blk_8763662564934652249': [5],
 'blk_-5861636720645142679': [5],
 'blk_7453815855294711849': [5],
 'blk_-28342503914935090': [5],
 'blk_8291449241650212794': [5],
 'blk_-5319073033164653435': [5],
 'blk_-4980916519894289629': [5],
 'blk_-5974833545991408899': [6],
 'b

In [55]:
len(log2index)

14

In [56]:
len(index2embed)

14